### calculated all the benchmarking metrics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc

import sys
sys.path.append("./")
import match
import metrics
import utils
from scipy.io import mmread
import os

sys.executable

'/home/bkzhu/python/miniconda3/envs/super_mario_testing/bin/python3'

In [2]:
meta = pd.read_csv('/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_meta.csv')
annt = meta['celltype'].to_numpy()

In [4]:
msx = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/maestro/rna_embed.csv")
msy = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/maestro/atac_embed.csv")

ms_dist = utils.cdist_correlation(msy.to_numpy(), msx.to_numpy())
ms_full_match, ms_scores = metrics.get_knn_matching(ms_dist)
msmatch = [np.arange(msx.shape[0]),ms_full_match,ms_scores]

In [5]:
scjx = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/scjoint/raw_labels/rna_embd.csv")
scjy = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/scjoint/raw_labels/atac_embd.csv")
scj_dist = utils.cdist_correlation(scjy.to_numpy(), scjx.to_numpy())
scj_full_match, scj_scores = metrics.get_knn_matching(scj_dist)
scjmatch = [np.arange(scjx.shape[0]),scj_full_match,scj_scores]

In [6]:
glx = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/glue/gl_rna.csv")
gly = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/glue/gl_atac.csv")

gl_dist = utils.cdist_correlation(gly.to_numpy(), glx.to_numpy())
gl_full_match, gl_scores = metrics.get_knn_matching(gl_dist)
glmatch = [np.arange(scjx.shape[0]),gl_full_match,gl_scores]

In [7]:
mf = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_idx_21.csv")
mfmatch = [mf['idx1'].tolist(),mf['idx2'].tolist(),mf['score'].tolist()]

In [10]:
order = (2, 1)
acc_ann_ms = metrics.get_matching_acc(
        matching=msmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_ms

0.5015197568389058

In [11]:
order = (2, 1)
acc_ann_scj = metrics.get_matching_acc(
        matching=scjmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_scj

0.6316374756741729

In [12]:
order = (2, 1)
acc_ann_mf = metrics.get_matching_acc(
        matching=mfmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_mf

0.6948001336154103

In [13]:
order = (2, 1)
acc_ann_gl = metrics.get_matching_acc(
        matching=glmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_gl

0.6726895734597157

In [14]:
ann_listlv1 = []
ann_listlv1.extend([acc_ann_mf, acc_ann_gl, acc_ann_scj, acc_ann_ms])
ann_listlv1

[0.6948001336154103,
 0.6726895734597157,
 0.6316374756741729,
 0.5015197568389058]

## start calculate metrics

In [15]:
mfx = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_embed_x0.csv")
mfy = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_embed_y0.csv")
mf_dist = utils.cdist_correlation(mfx.to_numpy(), mfy.to_numpy())

In [17]:
mf_fos = metrics.get_foscttm(mf_dist)
scj_fos = metrics.get_foscttm(scj_dist)
ms_fos = metrics.get_foscttm(ms_dist)
gl_fos = metrics.get_foscttm(gl_dist)

In [18]:
foscttm_list = []
foscttm_list.extend([mf_fos, gl_fos, scj_fos, ms_fos])
foscttm_list

[0.028984285956418086,
 0.04944216508441077,
 0.10427051697715503,
 0.13248661226825573]

In [19]:
slt_f1 = []
ari_f1 = []

mf_int = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/mf/metrics.csv")
ms_int = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/maestro/metrics.csv")
gl_int = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/glue/metrics.csv")
scj_int = pd.read_csv("/atac_bench_nrz/greanleaf_cortical/scjoint/metrics.csv")

slt_f1.extend([mf_int.loc[0,'slt_f1'], gl_int.loc[0,'slt_f1'], scj_int.loc[0,'slt_f1'], ms_int.loc[0,'slt_f1']])
ari_f1.extend([mf_int.loc[0,'ari_f1'], gl_int.loc[0,'ari_f1'], scj_int.loc[0,'ari_f1'], ms_int.loc[0,'ari_f1']])

slt_f1

[0.5054028608503665,
 0.5261296261543288,
 0.5391149738594236,
 0.4960443204287383]

In [20]:
ari_f1

[0.5794087451939259, 0.5874955638600893, 0.602003226808822, 0.5791055355221286]

In [21]:
m = ["mf","gl","scj","ms"]
minfo = m

data = {'method':minfo,'slt_f1': slt_f1, 'ari_f1':ari_f1,
       'ann1':ann_listlv1, 'foscttm':foscttm_list}

matching_result = pd.DataFrame(data)
matching_result

,method,slt_f1,ari_f1,ann1,foscttm
0,mf,0.505403,0.579409,0.694800,0.028984
1,gl,0.526130,0.587496,0.672690,0.049442
2,scj,0.539115,0.602003,0.631637,0.104271
3,ms,0.496044,0.579106,0.501520,0.132487


In [22]:
matching_result.to_csv("/atac_bench_nrz/greanleaf_cortical/metric_results.csv")

## KNN

In [23]:
# knn search tmp
k = 100
knn_match_mf = metrics.get_knn_alignment_score(
    dist=mf_dist,
    k_max=k
)
knn_match_scj = metrics.get_knn_alignment_score(
    dist=scj_dist,
    k_max=k
)
knn_match_ms = metrics.get_knn_alignment_score(
    dist=ms_dist,
    k_max=k
)
knn_match_gl = metrics.get_knn_alignment_score(
    dist=gl_dist,
    k_max=k
)

In [24]:
data = {'mf': knn_match_mf,
        'scj':knn_match_scj,
        'ms':knn_match_ms,
        'gl':knn_match_gl#,
       }
knn_tmp = pd.DataFrame(data)
knn_tmp

,mf,scj,ms,gl
0,0.021601,0.002895,0.003563,0.013139
1,0.032958,0.005345,0.006347,0.022381
2,0.043091,0.007906,0.008017,0.032402
3,0.053223,0.010244,0.009910,0.040196
4,0.061463,0.011803,0.011914,0.047099
...,...,...,...,...
95,0.417771,0.129718,0.157778,0.340719
96,0.420109,0.130943,0.158780,0.342055
97,0.422336,0.131945,0.160116,0.343614
98,0.425899,0.132836,0.161675,0.345284


In [26]:
import numpy as np

knn_tmp = []
knn_tmp.extend([knn_match_mf.tolist(), knn_match_gl.tolist(),
                knn_match_scj.tolist(), knn_match_ms.tolist()])
m = np.array(["mf", "gl","scj","ms"])
minfo = np.repeat(m, [100,100,100,100], axis=0).tolist()
minfo2 = minfo #* 5
knn = [item for sublist in knn_tmp for item in sublist]
step = [i for i in range(100)]*4 #*5
data = {'method':minfo2,'knn_tmp': knn, 'step':step }

knn_result = pd.DataFrame(data)
knn_result

,method,knn_tmp,step
0,mf,0.021601,0
1,mf,0.032958,1
2,mf,0.043091,2
3,mf,0.053223,3
4,mf,0.061463,4
...,...,...,...
395,ms,0.157778,95
396,ms,0.158780,96
397,ms,0.160116,97
398,ms,0.161675,98


In [27]:
# save the results as csvs
knn_result.to_csv("/atac_bench_nrz/greanleaf_cortical/knntmp.csv")